In [ ]:
from urllib.parse import urlparse# %%
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.alert import Alert
import time
from bs4 import BeautifulSoup
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import csv
import os
import requests
from selenium.webdriver.support.ui import Select
from datetime import datetime
import os
from selenium.webdriver.support.ui import Select
import csv
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.remote.webelement import WebElement
import openpyxl
from collections import Counter
import re
import pandas as pd
from selenium.common.exceptions import TimeoutException
import json
from urllib.parse import quote
from lxml import etree
import json
# from openai import OpenAI
import json
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse
# import cloudscraper
# from bs4 import BeautifulSoup
from lxml import html as html_parser
import requests
import os

# loading data from .env file
from dotenv import load_dotenv
import pyautogui

# import data combinations
from term_life_data import *
# from whole_life_data import *
# from endowment_products_data import *

def get_driver():
    options = Options()
    # options.add_experimental_option("debuggerAddress", "127.0.0.1:9224")
    # options.add_experimental_option("excludeSwitches", ["enable-automation"])
    # options.add_experimental_option("useAutomationExtension", False)

    # open specific chrome proofile
    profile_path=os.path.join(os.getcwd(),"profile")
    options.add_argument(f"user-data-dir={profile_path}")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.199 Safari/537.36")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument(f"--log-level=3")
    options.add_argument("start-maximized")
    driver = webdriver.Chrome(options=options)
    # driver = webdriver.Chrome(options=options)
    action = ActionChains(driver)
    return driver, action


def switch_to_window(url_part,driver):
    for handle in driver.window_handles:
        driver.switch_to.window(handle)
        if url_part in driver.current_url:
            break

driver,action= get_driver()

In [ ]:
data = []

class TermLifeAutomation:
    def __init__(self, search_data):
        self.search_data = search_data

    def do_search(self):
        print("Setting search options for Term Life products")
        self.search_term_life()
        print("Searching is done...")

    def filter_specific_providers(self):        
        # select only specific providers
        providers_selectors = [
            '[title="China Taiping Insurance (Singapore) Pte. Ltd."]',
            '[title="Manulife (Singapore) Pte. Ltd."]',
            '[title="Tokio Marine Life Insurance Singapore Ltd"]',
            '[title="China Life Insurance (Singapore) Pte. Ltd."]',
            '[title="Income Insurance Limited"]',
            '[title="Singapore Life Ltd."]'
        ]

        # first unchecking the box
        uncheck_all = self.wait_element(By.CSS_SELECTOR, '[id="AllInsurersCHK"]', 20)
        uncheck_all.click()

        self.wait_element(By.CSS_SELECTOR, '[class="search-head"]', 20).click()

        for provider_selector in providers_selectors:
            ele = self.wait_element(using=By.CSS_SELECTOR, css_code=provider_selector, timeout=2)        
            if ele:
                checkbox = ele.find_element(By.XPATH, "..//input[@type='checkbox']")
                checkbox.click()

    def search_term_life(self):
        while True:
            try:
                driver.get("https://www.comparefirst.sg/wap/homeEvent.action")
                time.sleep(2)
                self.wait_element(By.XPATH, '//label[contains(text(),"Term Life products")]', 20)
                break
            except:
                continue

        # clicking on specific products type section
        self.wait_element(By.XPATH, '//label[contains(text(),"Term Life products")]', 20).click()
        
        self.wait_element(By.CSS_SELECTOR, '[data-id="non-bips"]', 20).click()

        # filling the date box
        dob_box = self.wait_element(By.CSS_SELECTOR, '[name="dob"]', 20)
        dob_box.clear()
        # calculating the dob based on age
        dob = datetime.now().year - self.search_data["age"]
        dob_box.send_keys(f"01/01/{dob}")

        # filling gender
        self.wait_element(By.CSS_SELECTOR, f'[data-id="{self.search_data["gender"]}"]', 20).click()

        # clicking smoker `yes`
        self.wait_element(By.CSS_SELECTOR, f'[data-id="{self.search_data["smoker"]}"]', 20).click()

        # selecting coverage term
        coverage_term_ele = self.wait_element(By.CSS_SELECTOR, '.coverageTermTLNonDCIPs a.select2-choice', 20)
        coverage_term_ele.click()
        time.sleep(0.2)
        coverage_term_text = self.search_data["coverage_term"]
        option = self.wait_element(By.XPATH, f'//div[contains(text(),"{coverage_term_text}")]', 20)
        option.click()

        # selecting sum assured
        sum_assured_text = self.search_data["sum_assured"]
        sum_assured_ele = self.wait_element(By.CSS_SELECTOR, '.SANonDCIPTermSingle a.select2-choice', 20)
        sum_assured_ele.click()
        time.sleep(0.2)
        option = self.wait_element(By.XPATH, f'//div[contains(text(),"{sum_assured_text}")]', 20)
        option.click()

        # clicking on critical illness button
        self.wait_element(By.CSS_SELECTOR, f'#cretical-ill [data-id="{self.search_data["critical_illness"]}"]', 20).click()

        # clicking on sort A-Z button
        sorting_ele = self.wait_element(By.CSS_SELECTOR, '.sortNonWLGroup a.select2-choice', 20)
        sorting_ele.click()
        time.sleep(0.2)
        option = self.wait_element(By.XPATH, '//div[contains(text(),"Insurer (A - Z)")]', 20)
        option.click()

        # clicking on search button
        self.wait_element(By.CSS_SELECTOR, '.btn-search', 20).click()

        # clicking on `i agree`
        self.wait_element(By.CSS_SELECTOR, '[id="iUnderstant"]', 20).click()
    
    def scroll_container(self):
        products_container = self.wait_element(By.CSS_SELECTOR, '#result_container', 20)
        
        # Initial scroll position
        last_height = driver.execute_script("return arguments[0].scrollHeight", products_container)
        scroll_attempts = 0
        max_attempts = 10  # Limit tries to prevent infinite loops

        while scroll_attempts < max_attempts:
            # Scroll down to bottom
            scroll_script = "arguments[0].scrollTo(0, arguments[0].scrollHeight);"
            driver.execute_script(scroll_script, products_container)
            
            # Wait to load page
            time.sleep(0.2)
            
            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return arguments[0].scrollHeight", products_container)
            
            # If heights are the same, we've reached the bottom or no new content is loading
            if new_height == last_height:
                break
                
            last_height = new_height
            scroll_attempts += 1

        print(f"Finished scrolling after {scroll_attempts} attempts")

    def check_if_provider_is_required(self, provider_name):
        print("checking provider: ", provider_name)
        all_providers = [
            "China Life Insurance (Singapore) Pte. Ltd.",
            "China Taiping Insurance (Singapore) Pte. Ltd.",
            "Income Insurance Limited",
            "Manulife (Singapore) Pte. Ltd.",
            "Singapore Life Ltd.",
            "Tokio Marine Life Insurance Singapore Ltd"
        ]
        
        return provider_name in all_providers
    
    def check_if_product_is_required(self, product_name):
        print("checking product: ", product_name)
        all_products = [
            "China Life Term Guardian",
            "i-Protect",
            "i-Protect (Renewable)",
            "Star Term Protect",
            "TermLife Solitaire",
            "ManuProtect Term (II) (with TPD Plus) (Level & Convertible)",
            "ManuProtect Term (II) (with TPD Plus) (Renewable & Convertible)",
            "Singlife Elite Term II",
            "Singlife Simple Term",
            "TM Term Assure (II)(Level & Convertible)",
            "TM Term Assure (II)(Renewable & Convertible)"
        ]
        return product_name in all_products
        
    def scrape_products(self):
        global data

        try:
            # getting total products available to scrape it looks like `showing 5 of 25`
            total_products_text = self.wait_element(By.CSS_SELECTOR, '[id="showingProducts"]', 20).text
            total_products = int(total_products_text.split(" ")[-1])
        except:
            return None

        index = 0

        while index < total_products:
            print(f"scraping data for product {index + 1} of {total_products}")
            # scroll products box to load all products
            self.scroll_container()

            # getting all `view details` buttons
            try:
                view_details_buttons = self.wait_element(By.CSS_SELECTOR, '[class="search_detail"]', 20, wait_all=True)
            except Exception as e:
                print("No products found")
                break

            # first checking if provider is what we need to scrape
            providers = self.wait_element(By.CSS_SELECTOR, '.result_content_inner h3', 20, wait_all=True)
            provider_element = providers[index]
            provider = provider_element.get_attribute('textContent')
            # print("innerHTML:", provider_element.get_attribute('innerHTML'))
            # print("textContent:", provider_element.get_attribute('textContent'))
            # provider = self.wait_element(By.CSS_SELECTOR, '.result_content_inner h3', 20, wait_all=True)[index].text
            # print("checking provider nè: ", provider)
            products = self.wait_element(By.CSS_SELECTOR, '[id="sProdName"]', 20, wait_all=True)
            products_element = products[index]
            product = products_element.get_attribute('textContent')

            # Skip if provider or product is not required
            if not self.check_if_provider_is_required(provider) or not self.check_if_product_is_required(product):
                print(f"Skipping product {index + 1} - Provider: {provider}, Product: {product}")
                index += 1
                continue

            current_product_button = view_details_buttons[index]
            self.js_click(current_product_button)

            # extracting data
            try:
                provider = self.wait_element(By.CSS_SELECTOR, '.product_details_right h3', 20).text
            except:
                print(f"Failed to extract provider details for product {index + 1}")
                driver.back()
                time.sleep(1)
                index += 1
                continue

            # Skip if provider is not required (double-check)
            if not self.check_if_provider_is_required(provider):
                print(f"Skipping product {index + 1} - Provider: {provider}")
                driver.back()
                time.sleep(1)
                index += 1
                continue

            product = self.wait_element(By.CSS_SELECTOR, '.prod-desc', 20).text
            print(f"scraping data for {product} product")
            
            coverage_term = self.wait_element(By.CSS_SELECTOR, '.policy-coverage-term', 20, wait_all=True)[0].text
            premium_term = self.wait_element(By.CSS_SELECTOR, '.pay-period', 20, wait_all=True)[0].text
            main_plan_annual_premium_no_ci = self.wait_element(By.CSS_SELECTOR, '.annual-premium', 20, wait_all=True)[0].text
            total_premium_no_ci = self.wait_element(By.CSS_SELECTOR, '.tot-premium', 20, wait_all=True)[0].text
            distribution_cost_no_ci = self.wait_element(By.CSS_SELECTOR, '.tot-dist-cost', 20, wait_all=True)[0].text

            product_data = {
                "product_type": "Term Life products",
                "provider": provider,
                "product": product,
                "Age": self.search_data["age"],
                "Gender": self.search_data["gender"],
                "Smoker": self.search_data["smoker"],
                "Critical Illness benefit": self.search_data["critical_illness"],
                "Premium Type": "Annual",
                "Sum Assured": self.search_data["sum_assured"],
                "Coverage Term": coverage_term,
                "Premium Term": premium_term,
                "Main Plan Annual Premium (No CI)": main_plan_annual_premium_no_ci,
                "Total Premium (No CI)": total_premium_no_ci,
                "Distribution Cost (No CI)": distribution_cost_no_ci,
                "Main Plan Annual Premium (With CI)": "",
                "Total Premium (With CI)": "",
                "Distribution Cost (With CI)": "",
                "Payout Starting": "-"
            }

            print(product_data)
            data.append(product_data)
            self.save_to_excel()

            # go back
            driver.back()
            time.sleep(1)

            index += 1
            if index >= total_products:
                break

        return data
    
    def save_to_excel(self):
        global data

        print("innerHTML data:", data)
        df = pd.DataFrame(data)
        file_name = "Term_Life_Products.xlsx"
        try:
            df.to_excel(file_name, index=False)
            print(f"Excel file created: {file_name}")
        except PermissionError:
            # Handle the case where the file is open or locked
            timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
            backup_file_name = f"Term_Life_Products_{timestamp}.xlsx"
            df.to_excel(backup_file_name, index=False)
            print(f"Original file is locked. Saved as: {backup_file_name}")

    def js_click(self, element):
        driver.execute_script("arguments[0].click();", element)
        time.sleep(0.5)

    def wait_element(self, using, css_code, timeout, wait_all=False):
        if wait_all:
            WebDriverWait(driver, timeout).until(
                EC.presence_of_all_elements_located((using, css_code)))
            elements = driver.find_elements(using, css_code)
            return elements
        else:
            try:
                element = WebDriverWait(driver, timeout).until(
                    EC.presence_of_element_located((using, css_code))
                )
                return element
            except Exception as e:
                return None

product_type="Term Life products"


term_life_data=get_term_data()


count = 0
while count < len(term_life_data):
    data_combination = term_life_data[count]
    print(data_combination)
    bot = TermLifeAutomation(search_data=data_combination)
    bot.do_search()
    output = bot.scrape_products()

    if output is not None:
        print("-------------------------------------------------------------------------")
        print("increasing count to +1 coz in last iteration there was gateway problem. ")
        print("-------------------------------------------------------------------------")
        count += 1
                